In [44]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
import random
import tensorflow.keras.utils as utils
import pydot
from tensorflow.keras import regularizers

In [45]:
dataset_dir = "Datasets/superconducter/train.csv"

In [46]:
df = pd.read_csv(dataset_dir, index_col=None)
df = df.fillna(df.mean())
df

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,...,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,29.00
1,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,...,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606,26.00
2,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,...,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697,19.00
3,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,...,2.264286,2.213364,2.226222,1.368922,1.048834,1,1.100000,0.433013,0.440952,22.00
4,4,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.90607,27.848743,51.968828,...,2.242857,2.213364,2.206963,1.368922,1.096052,1,1.057143,0.433013,0.428809,23.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21258,4,106.957877,53.095769,82.515384,43.135565,1.177145,1.254119,146.88130,15.504479,65.764081,...,3.555556,3.223710,3.519911,1.377820,0.913658,1,2.168889,0.433013,0.496904,2.44
21259,5,92.266740,49.021367,64.812662,32.867748,1.323287,1.571630,188.38390,7.353333,69.232655,...,2.047619,2.168944,2.038991,1.594167,1.337246,1,0.904762,0.400000,0.212959,122.10
21260,2,99.663190,95.609104,99.433882,95.464320,0.690847,0.530198,13.51362,53.041104,6.756810,...,4.800000,4.472136,4.781762,0.686962,0.450561,1,3.200000,0.500000,0.400000,1.98
21261,2,99.663190,97.095602,99.433882,96.901083,0.690847,0.640883,13.51362,31.115202,6.756810,...,4.690000,4.472136,4.665819,0.686962,0.577601,1,2.210000,0.500000,0.462493,1.84


In [47]:
df_additional = pd.read_csv("Datasets/superconducter/unique_m.csv", index_col=None)
df_additional = df_additional.fillna(df.mean())
df_additional = df_additional.iloc[:,:-2]
df_additional

,H,He,Li,Be,B,C,N,O,F,Ne,...,Ir,Pt,Au,Hg,Tl,Pb,Bi,Po,At,Rn
0,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
1,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
2,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
3,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
4,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21258,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
21259,0.0,0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0
21260,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
21261,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0


In [48]:
label_column = df.pop('critical_temp')
label_column = np.asarray(label_column)
label_column = label_column - 1
df.insert(0, 'label', label_column)
df

,label,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,...,mean_Valence,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence
0,28.00,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,...,2.25,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059
1,25.00,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,...,2.00,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606
2,18.00,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,...,2.25,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697
3,21.00,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,...,2.25,2.264286,2.213364,2.226222,1.368922,1.048834,1,1.100000,0.433013,0.440952
4,22.00,4,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.90607,27.848743,...,2.25,2.242857,2.213364,2.206963,1.368922,1.096052,1,1.057143,0.433013,0.428809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21258,1.44,4,106.957877,53.095769,82.515384,43.135565,1.177145,1.254119,146.88130,15.504479,...,3.25,3.555556,3.223710,3.519911,1.377820,0.913658,1,2.168889,0.433013,0.496904
21259,121.10,5,92.266740,49.021367,64.812662,32.867748,1.323287,1.571630,188.38390,7.353333,...,2.20,2.047619,2.168944,2.038991,1.594167,1.337246,1,0.904762,0.400000,0.212959
21260,0.98,2,99.663190,95.609104,99.433882,95.464320,0.690847,0.530198,13.51362,53.041104,...,4.50,4.800000,4.472136,4.781762,0.686962,0.450561,1,3.200000,0.500000,0.400000
21261,0.84,2,99.663190,97.095602,99.433882,96.901083,0.690847,0.640883,13.51362,31.115202,...,4.50,4.690000,4.472136,4.665819,0.686962,0.577601,1,2.210000,0.500000,0.462493


In [49]:
df = pd.concat([df, df_additional], axis=1)
df

,label,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,...,Ir,Pt,Au,Hg,Tl,Pb,Bi,Po,At,Rn
0,28.00,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
1,25.00,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
2,18.00,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
3,21.00,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
4,22.00,4,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.90607,27.848743,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21258,1.44,4,106.957877,53.095769,82.515384,43.135565,1.177145,1.254119,146.88130,15.504479,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
21259,121.10,5,92.266740,49.021367,64.812662,32.867748,1.323287,1.571630,188.38390,7.353333,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0
21260,0.98,2,99.663190,95.609104,99.433882,95.464320,0.690847,0.530198,13.51362,53.041104,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
21261,0.84,2,99.663190,97.095602,99.433882,96.901083,0.690847,0.640883,13.51362,31.115202,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0


In [50]:
df.isnull().values.any()

False

In [51]:
df = df.sample(frac=1, random_state=41)

In [52]:
col_num = len(df.columns)
row_num = len(df.index)
print("Number of columns is: {} and number of rows is: {}".format(col_num, row_num))

Number of columns is: 168 and number of rows is: 21263


In [53]:
trainx = df.iloc[:int(0.8*row_num), 1:(col_num)]
trainy = df.iloc[:int(0.8*row_num), 0]

testx = df.iloc[int(0.8*row_num):, 1:(col_num)]
testy = df.iloc[int(0.8*row_num):, 0]

In [54]:
trainy

8801      17.000
9916      26.500
3983     119.000
1747      21.200
18154      5.500
          ...   
16397     -0.859
20076      3.370
197       87.000
6366       3.400
12100      3.230
Name: label, Length: 17010, dtype: float64

In [55]:
scaler = StandardScaler()
trainx_scaled = pd.DataFrame(scaler.fit_transform(trainx), columns = trainx.columns, index = trainx.index)
textx_scaled = pd.DataFrame(scaler.transform(testx), columns = testx.columns, index = testx.index)

In [56]:
trainx_scaled

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,Ir,Pt,Au,Hg,Tl,Pb,Bi,Po,At,Rn
8801,0.622770,0.559197,-0.389294,0.271289,-0.598997,0.739733,0.692870,0.469472,-0.215191,0.531020,...,-0.071819,-0.109166,-0.030657,-0.175072,-0.174210,-0.151436,-0.308529,0.0,0.0,0.0
9916,2.011786,-0.649589,-0.894676,-0.398791,-0.718203,1.832455,1.453853,-0.348755,-0.750733,-0.714514,...,-0.071819,-0.109166,-0.030657,-0.175072,-0.174210,-0.151436,-0.308529,0.0,0.0,0.0
3983,1.317278,0.780030,-0.574041,0.227552,-0.645712,1.013571,1.566360,1.380965,-0.794773,1.554286,...,-0.071819,-0.109166,-0.030657,2.750529,-0.174210,1.257129,-0.308529,0.0,0.0,0.0
1747,0.622770,-0.493824,-0.485967,-0.392693,-0.623522,0.772557,-0.158855,0.134719,0.120244,-0.204388,...,-0.071819,-0.109166,-0.030657,-0.175072,-0.174210,-0.151436,-0.308529,0.0,0.0,0.0
18154,-0.766245,-0.005136,0.476024,0.360930,0.635773,-0.330921,-0.899756,-0.798692,0.888435,-0.719239,...,-0.071819,-0.109166,-0.030657,-0.175072,-0.174210,-0.151436,-0.308529,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16397,-0.766245,3.120916,2.809812,3.373736,2.832486,-0.236505,0.081762,-0.623046,-0.763466,-0.349883,...,-0.071819,-0.109166,-0.030657,-0.175072,3.524897,-0.151436,1.215128,0.0,0.0,0.0
20076,0.622770,-0.119983,0.415299,-0.708710,-0.077067,0.034309,0.240657,1.413454,0.255665,1.698265,...,-0.071819,-0.109166,-0.030657,-0.175072,-0.174210,-0.151436,1.215128,0.0,0.0,0.0
197,0.622770,-0.568692,0.200449,-0.467811,0.206844,0.739338,0.516126,0.105915,0.135355,-0.175551,...,-0.071819,-0.109166,-0.030657,-0.175072,-0.174210,-0.151436,-0.308529,0.0,0.0,0.0
6366,0.622770,0.471574,-0.503174,0.226256,-0.598801,0.752614,0.971001,0.343742,-0.564029,0.423534,...,-0.071819,-0.109166,-0.030657,-0.175072,-0.174210,-0.151436,-0.308529,0.0,0.0,0.0


In [57]:
import random
def create_rand_list(max_val, count):
    randomlist = random.sample(range(0, max_val + 1), count)
    return randomlist

In [58]:
def get_MICS_model(inp_size, drop_out, hidden_num = 12, hidden_size=256):
    inputs = keras.layers.Input(shape=(inp_size), name="input")
        
    h = keras.layers.Dense(hidden_size, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(inputs)
    h = keras.layers.Dropout(drop_out)(h)
    for hidden in range(hidden_num):
        h = keras.layers.Dense(hidden_size, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(h)
        h = keras.layers.Dropout(drop_out)(h) 

    outputs = keras.layers.Dense(1, activation="relu")(h)    
    return keras.Model(inputs=[inputs], outputs = outputs)

In [59]:
df["label"].value_counts()

 79.0000    143
 19.0000    129
 29.0000    125
 89.0000    122
 39.0000    111
           ... 
 14.1400      1
-0.6050       1
 45.1800      1
 12.8200      1
-0.9318       1
Name: label, Length: 3007, dtype: int64

In [61]:
inp_size = len(trainx.columns)
MICS_model = get_MICS_model(inp_size, drop_out = 0.25)
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=25), 
        keras.callbacks.ReduceLROnPlateau("val_loss", factor = 0.5, patience=10,
                                         verbose = 2, mode = "auto", 
                                          min_lr = 1e-6)]
MICS_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=keras.losses.MeanSquaredError())
history = MICS_model.fit(x = [trainx_scaled], y = trainy.values,  
                         validation_data = ([textx_scaled], testy.values),
                         epochs=300, batch_size = 32, callbacks=callback)
training_val_accuracy = history.history["val_accuracy"]
best_row_index = np.argmax(training_val_accuracy)
best_val_accuracy = training_val_accuracy[best_row_index]
best_val_accuracy
#min_losses3[c].append(best_val_loss)
#i = i + 1

Epoch 1/300
532/532 [==============================] - 5s 9ms/step - loss: 502.1050 - val_loss: 380.3188
Epoch 2/300
532/532 [==============================] - 5s 10ms/step - loss: 338.7940 - val_loss: 274.8993
Epoch 3/300
532/532 [==============================] - 5s 10ms/step - loss: 268.8231 - val_loss: 212.2175
Epoch 4/300
532/532 [==============================] - 5s 10ms/step - loss: 256.5108 - val_loss: 239.0951
Epoch 5/300
532/532 [==============================] - 5s 10ms/step - loss: 244.0853 - val_loss: 219.0197
Epoch 6/300
532/532 [==============================] - 5s 10ms/step - loss: 224.4958 - val_loss: 181.9306
Epoch 7/300
532/532 [==============================] - 5s 10ms/step - loss: 214.3799 - val_loss: 182.6061
Epoch 8/300
532/532 [==============================] - 6s 10ms/step - loss: 208.1261 - val_loss: 199.0696
Epoch 9/300
532/532 [==============================] - 5s 10ms/step - loss: 211.1569 - val_loss: 190.2393
Epoch 10/300
532/532 [=========================

KeyError: 'val_accuracy'

In [19]:
k = MICS_model.predict(trainx_scaled)
k

array([[12.455019],
       [72.27619 ],
       [72.27619 ],
       ...,
       [72.27619 ],
       [72.27619 ],
       [12.455019]], dtype=float32)

In [104]:
k = pd.DataFrame(k)
k[0].value_counts()

0    2593
5    2301
Name: 0, dtype: int64